In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [3]:
import urllib.request  # lib that handles URLs
import io
import pandas as pd
pd.options.display.max_rows=250
import numpy as np
from datetime import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter

import pyspark.sql.functions as func

In [4]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline, Model
from pyspark.sql.functions import col

In [5]:
df_events = spark.read.parquet("s3://bigdataproject-pr/filtered_events.parquet")
df_events.cache()

DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: int, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]

In [6]:
relevant_events = df_events.where(df_events.SOURCEURL.rlike('trump'))


In [7]:
continuousCols = ["GoldsteinScale", "NumMentions", "NumSources", "NumArticles"]

for col in continuousCols:
    df_events = df_events.withColumn(col, func.col(col).cast("double"))

In [24]:
df_events = df_events.withColumn("label", df_events.AvgTone)

In [8]:
df_events = df_events.withColumn("MonthYear", func.col("MonthYear").cast("string"))
df_events_2017 = df_events.where(df_events.MonthYear.like("2017%"))
df_events_2018 = df_events.where(df_events.MonthYear.like("2018%"))


In [33]:
df_events.printSchema

<bound method DataFrame.printSchema of DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: date, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: double, NumSources: double, NumArticles: double, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]>

In [26]:
stringInputs = ["Actor1Name", "Actor2Name", "EventCode", "Actor1Geo_FullName", "Actor2Geo_FullName", 
                "ActionGeo_CountryCode","SOURCEURL"]


indexers = [ StringIndexer(inputCol=c, outputCol=c + "_IX", handleInvalid= "skip") 
            for c in stringInputs ]

encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers]

assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols, outputCol="features")



In [17]:
vector_assembler= VectorAssembler(
    inputCols=df_events.columns[0:-1]
    , outputCol='features'
)

In [11]:
type(df_events_2017)

pyspark.sql.dataframe.DataFrame

In [10]:
from pyspark.ml.feature import RFormula

formula = RFormula(formula = "AvgTone ~ .", featuresCol = 'features', labelCol = 'label')
fittedRF = formula.fit(df_events_2017)
preparedDF = fittedRF.transform(df_events_2017)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41143)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41143)

In [27]:

rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=10)

In [28]:
pipeline = Pipeline(stages=indexers + encoders + assembler, rf)

In [29]:
rf_model = pipeline.fit(df_events_2017)

TypeError: Cannot recognize a pipeline stage of type <class 'list'>.

In [31]:
spark.stop()